In [ ]:
%reload_ext setup

## Demo

In [ ]:
import asyncio
import os
import logging
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult

from igent.matching_agent import get_agents

# Suppress autogen_core and httpx INFO logs
logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [ ]:
import os
import json
from igent.tools import read_txt, read_json, save_json_tool  # Assuming save_json_tool is imported
import asyncio
from pathlib import Path

# Updated Prompts
a_matcher = await read_txt("../prompts/a_matcher.txt")
a_critic = await read_txt("../prompts/a_critic.txt")
b_matcher = await read_txt("../prompts/b_matcher.txt")
b_critic = await read_txt("../prompts/b_critic.txt")

# Workflow Orchestration
async def run_workflow(
    config_path: str,
    registrations_file: str = "registrations.json",
    offers_file: str = "offers.json",
    incentives_file: str = "incentives.json",
    matches_file: str = "matches.json",
    final_matches_file: str = "final_matches.json"
):
    # Load registrations once
    registrations_data = await read_json(registrations_file)
    if not isinstance(registrations_data, list):
        print("Error: Registrations file must contain a list.")
        return
    
    # Process up to 10 registrations
    max_items = min(10, len(registrations_data))
    if max_items == 0:
        print("No registrations to process.")
        return

    print(f"Processing {max_items} registrations...")
    
    for i in range(max_items):
        print(f"\nProcessing registration {i+1}/{max_items}")
        
        # Pop the first registration
        current_registration = registrations_data.pop(0)
        
        # Update the registrations file with remaining items
        abs_registrations_path = os.path.abspath(registrations_file)
        Path(abs_registrations_path).parent.mkdir(parents=True, exist_ok=True)
        with open(abs_registrations_path, "w") as f:
            json.dump(registrations_data, f, indent=2)

        # Pair 1: Matcher + Critic
        pair1 = await get_agents(
            config_path=config_path,
            matcher_prompt=a_matcher,
            critic_prompt=a_critic
        )
        
        message1 = (
            f"Match registrations to suppliers:\n"
            f"REGISTRATION: ```{[current_registration]}```\n"  # Single item as list
            f"OFFERS: ```{await read_json(offers_file)}```\n"
            f"On approval, SAVE the output to '{matches_file}' using save_json_tool."
        )
        
        print(f"Running Pair 1 for registration {current_registration.get('registration_id', 'unknown')}")
        success1 = False
        async for msg in pair1.run_stream(
            task=[TextMessage(content=message1, source="user")],
            cancellation_token=CancellationToken()
        ):
            if isinstance(msg, TextMessage):
                if msg.source == "user":
                    print(f"{msg.source}: {msg.content[:100]}")
                else:
                    print(f"{msg.source}: {msg.content}")
            elif isinstance(msg, TaskResult):
                final_message = "Pair 1 completed. "
                if msg.stop_reason:
                    final_message += f"Stop reason: {msg.stop_reason}"
                    if "APPROVE" in msg.stop_reason:
                        success1 = True
                print(final_message)
        
        if not success1:
            print(f"Pair 1 failed for registration {i+1}. Aborting.")
            continue
        
        await asyncio.sleep(0.1)  # Ensure file write completes
        if not os.path.exists(matches_file):
            print(f"Error: '{matches_file}' was not saved for registration {i+1}. Aborting.")
            return

        # Pair 2: Subsidy Matcher + Subsidy Critic
        pair2 = await get_agents(
            config_path=config_path,
            matcher_prompt=b_matcher,
            critic_prompt=b_critic
        )
        
        message2 = (
            f"Enrich matches with pricing and subsidies:\n"
            f"MATCHES: ```{await read_json(matches_file)}```\n"
            f"OFFERS: ```{await read_json(offers_file)}```\n"
            f"INCENTIVES: ```{await read_json(incentives_file)}```\n"
            f"On approval, SAVE the output to '{final_matches_file}' using save_json_tool."
        )
        
        print(f"\nRunning Pair 2 for registration {current_registration.get('registration_id', 'unknown')}")
        success2 = False
        async for msg in pair2.run_stream(
            task=[TextMessage(content=message2, source="user")],
            cancellation_token=CancellationToken()
        ):
            if isinstance(msg, TextMessage):
                if msg.source == "user":
                    print(f"{msg.source}: {msg.content[:100]}")
                else:
                    print(f"{msg.source}: {msg.content}")
            elif isinstance(msg, TaskResult):
                final_message = "Pair 2 completed. "
                if msg.stop_reason:
                    final_message += f"Stop reason: {msg.stop_reason}"
                    if "APPROVE" in msg.stop_reason:
                        success2 = True
                print(final_message)
        
        if not success2:
            print(f"Pair 2 failed for registration {i+1}. Continuing to next registration.")
            continue
        
        await asyncio.sleep(0.1)  # Ensure file write completes
        if not os.path.exists(final_matches_file):
            print(f"Error: '{final_matches_file}' was not saved for registration {i+1}. Continuing.")
            continue

    print(f"\nProcessed {max_items} registrations successfully.")

# Run the workflow
await run_workflow(
    config_path="../static/model_config.yaml",
    registrations_file="../data/demo/mock_registrations.json",
    offers_file="../data/demo/offers.json",
    incentives_file="../data/demo/incentives.json",
    matches_file="../data/demo/matches.json",
    final_matches_file="../data/demo/pos.json"
)

## Sandbox

In [ ]:
%reload_ext setup


In [ ]:
from igent.tools import pop_json

await pop_json(source_file="../data/demo/registrations.json", popped_file="../data/demo/popped.json")

In [ ]:
from igent.tools import update_supplier_capacity
data = """
[  {
    "registration_id": "SPUS52157",
    "campaign_name": "Minnesota 1 Summer 2024",
    "supplier_id": "iSoalr",
    "zip_code": "55409",
    "panel_name": "Silfab HG",
    "panel_capacity": "410",
    "battery_name": null,
    "battery_capacity": null,
    "num_panels": "12",
    "product_type": "Solar",
    "product_price": "15444",
    "eligible_subsidies": [
      "Federal Tax Credit"
    ],
    "payment_type": "Cash"
  }
]
"""
await update_supplier_capacity(data, "../data/demo/offers.json")